In [246]:
import sqlite3
import pandas as pd

con = sqlite3.connect("database.sqlite")

In [247]:
# country_id="21518" and league_id="21518" define La Liga
# id="43040" or team_api_id="8633" define Real Madrid
# id="43039" or team_api_id="9783" define RC Deportivo de La Coruña (opponent of Real Madrid in stage 38)
df = pd.read_sql_query('SELECT stage, date, home_team_goal, away_team_goal, '
                       'home_team_api_id, away_team_api_id, '
                       'B365H, B365D, B365A '
                       'FROM Match '
                       'WHERE country_id="21518" '
                       'AND league_id="21518" '
                       'ORDER BY season, stage ASC', con)

In [248]:
team = 8633

In [249]:
teams = df['home_team_api_id'].unique()
points_against_teams = {}
points_of_team = {}
clashes = {}
appearances = {}
for team_id in teams:
    points_against_teams[team_id] = {}
    clashes[team_id] = {}
    points_of_team[team_id] = []
    appearances[team_id] = 0
    for team_id_inner in teams:
        if team_id != team_id_inner:
            points_against_teams[team_id][team_id_inner] = []
            clashes[team_id][team_id_inner] = 0
for i, value in enumerate(df.values):
    home_team_api_id = df['home_team_api_id'].values[i]
    away_team_api_id = df['away_team_api_id'].values[i]
    home_team_goal = df['home_team_goal'].values[i]
    away_team_goal = df['away_team_goal'].values[i]
    if home_team_goal > away_team_goal:
        points_against_teams[home_team_api_id][away_team_api_id].append(3)
        points_against_teams[away_team_api_id][home_team_api_id].append(0)
        points_of_team[home_team_api_id].append(3)
        points_of_team[away_team_api_id].append(0)
    elif home_team_goal < away_team_goal:
        points_against_teams[home_team_api_id][away_team_api_id].append(0)
        points_against_teams[away_team_api_id][home_team_api_id].append(3)
        points_of_team[home_team_api_id].append(0)
        points_of_team[away_team_api_id].append(3)
    else:
        points_against_teams[home_team_api_id][away_team_api_id].append(1)
        points_against_teams[away_team_api_id][home_team_api_id].append(1)
        points_of_team[home_team_api_id].append(1)
        points_of_team[away_team_api_id].append(1)


In [250]:
bets_of_team = []
enemies_of_team = []
wins_of_team_against_enemy_in_last_five = []
wins_of_team_in_last_five = []
points_of_team_in_last_five = []
for i, value in enumerate(df.values):
    home_team_api_id = df['home_team_api_id'].values[i]
    away_team_api_id = df['away_team_api_id'].values[i]
    if home_team_api_id == team:
        enemies_of_team.append(df['away_team_api_id'].values[i])
        bets_of_team.append(df['B365H'].values[i])
        wins_against = 0
        wins = 0
        points = 0
        for j in range(clashes[home_team_api_id][away_team_api_id] - 5, clashes[home_team_api_id][away_team_api_id]):
            if j >= 0 and points_against_teams[home_team_api_id][away_team_api_id][j] == 3:
                wins_against += 1
        for j in range(appearances[home_team_api_id] - 5, appearances[home_team_api_id]):
            if j >= 0 and points_of_team[home_team_api_id][j] == 3:
                wins += 1
            points += points_of_team[home_team_api_id][j]
        wins_of_team_against_enemy_in_last_five.append(wins_against)
        wins_of_team_in_last_five.append(wins)
        points_of_team_in_last_five.append(points)
        clashes[home_team_api_id][away_team_api_id] += 1
        clashes[away_team_api_id][home_team_api_id] += 1
        appearances[home_team_api_id] += 1
    if away_team_api_id == team:
        enemies_of_team.append(df['home_team_api_id'].values[i])
        bets_of_team.append(df['B365A'].values[i])
        wins_against = 0
        wins = 0
        points = 0
        for j in range(clashes[away_team_api_id][home_team_api_id] - 5, clashes[away_team_api_id][home_team_api_id]):
            if j >= 0 and points_against_teams[away_team_api_id][home_team_api_id][j] == 3:
                wins_against += 1
        for j in range(appearances[away_team_api_id] - 5, appearances[away_team_api_id]):
            if j >= 0 and points_of_team[away_team_api_id][j] == 3:
                wins += 1
            points += points_of_team[away_team_api_id][j]
        wins_of_team_against_enemy_in_last_five.append(wins_against)
        wins_of_team_in_last_five.append(wins)
        points_of_team_in_last_five.append(points)
        clashes[home_team_api_id][away_team_api_id] += 1
        clashes[away_team_api_id][home_team_api_id] += 1
        appearances[away_team_api_id] += 1
#wins_of_team_in_last_five = [number / 5 for number in wins_of_team_in_last_five]
#wins_of_team_against_enemy_in_last_five = [number / 5 for number in wins_of_team_against_enemy_in_last_five]
#points_of_team_in_last_five = [number / 15 for number in points_of_team_in_last_five]

In [251]:
import pymc3 as pm
import arviz as az

class GaussianModel:
    def __init__(self, wins_team, wins_team_against, points_team, bets):
        self.wins_team = wins_team
        self.wins_team_against = wins_team_against
        self.points_team = points_team
        self.bets = bets


    def run(self):
        basic_model = pm.Model()
        with basic_model:
            p_wins_team_against = pm.Normal("p_wins_team_against", mu=0, sigma=1)
            p_wins_team = pm.Normal("p_wins_team", mu=0, sigma=1)
            p_points_team = pm.Normal("p_points_team", mu=0, sigma=1)
            mu = p_wins_team_against * self.wins_team_against +\
                 p_points_team * self.points_team +\
                 p_wins_team * self.wins_team
            pm.Normal("bets", mu=mu, sigma=1, observed=self.bets)
            self.trace = pm.sample(5000, tune=2000)
            az.plot_trace(self.trace, show=True)
            print(az.summary(self.trace, round_to=2))

In [252]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch
import torch.nn as nn
from pyro.infer.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO, Predictive
from tqdm.auto import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np

In [253]:
class BayesianRegression(PyroModule):
    def __init__(self, input_features=2, h1=20, h2=20):
        super().__init__()
        self.input_features=input_features
        self.fc1 = PyroModule[nn.Linear](input_features, h1)
        self.fc1.weight = PyroSample(dist.Normal(0., 1.).expand([h1, input_features]).to_event(2))
        self.fc1.bias = PyroSample(dist.Normal(0., 1.).expand([h1]).to_event(1))
        self.fc2 = PyroModule[nn.Linear](h1, h2)
        self.fc2.weight = PyroSample(dist.Normal(0., 1.).expand([h2, h1]).to_event(2))
        self.fc2.bias = PyroSample(dist.Normal(0., 1.).expand([h2]).to_event(1))
        self.fc3 = PyroModule[nn.Linear](h2, 1)
        self.fc3.weight = PyroSample(dist.Normal(0., 1.).expand([1, h2]).to_event(2))
        self.fc3.bias = PyroSample(dist.Normal(0., 1.).expand([1]).to_event(1))
        self.relu = nn.ReLU()

    def forward(self, x, y=None):
        x = x.reshape(-1, self.input_features)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = self.fc3(x).squeeze()
        sigma = pyro.sample("sigma", dist.Uniform(0., 1.))
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma), obs=y)
        return mu

In [254]:
model = BayesianRegression()
guide = AutoDiagonalNormal(model)
adam = pyro.optim.Adam({"lr": 1e-3})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [255]:
x_data = torch.tensor([np.array(wins_of_team_in_last_five[50:250]), np.array(wins_of_team_against_enemy_in_last_five[50:250])], dtype=torch.float)
x_data_test = torch.from_numpy(np.array(wins_of_team_in_last_five[250:304])).float()
y_data = torch.from_numpy(np.array(bets_of_team[50:250])).float()

In [256]:
num_iterations = 2000
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(x_data)))

[iteration 0001] loss: 1296.0164
[iteration 0101] loss: 571.6701
[iteration 0201] loss: 679.6572
[iteration 0301] loss: 558.3608
[iteration 0401] loss: 830.2454
[iteration 0501] loss: 565.8932
[iteration 0601] loss: 499.9296
[iteration 0701] loss: 486.0937
[iteration 0801] loss: 471.9117
[iteration 0901] loss: 472.9617
[iteration 1001] loss: 465.0841
[iteration 1101] loss: 460.3493
[iteration 1201] loss: 460.2022
[iteration 1301] loss: 430.6362
[iteration 1401] loss: 485.4915
[iteration 1501] loss: 414.1415
[iteration 1601] loss: 422.8718
[iteration 1701] loss: 457.2437
[iteration 1801] loss: 494.2579
[iteration 1901] loss: 427.1120


In [257]:
predictive = Predictive(model, guide=guide, num_samples=500)
preds = predictive(x_data_test)

y_pred = preds['obs'].T.detach().numpy().mean(axis=1)
y_std = preds['obs'].T.detach().numpy().std(axis=1)

In [258]:
error = 0
for i, bet in enumerate(y_pred):
    error += abs(bet - bets_of_team[250 + i])
avg_error = error / 54
print(avg_error)

0.20285174519927415


In [259]:
#
#gauss = GaussianModel(wins_team_against=wins_of_team_against_enemy_in_last_five,
#                      wins_team=wins_of_team_in_last_five,
#                      points_team=points_of_team_in_last_five,
#                      bets=bets_of_team)
#gauss.run()
#
##%%
#
#mean_wins_team_against = gauss.trace["p_wins_team_against"].mean()
#mean_wins_team = gauss.trace["p_wins_team"].mean()
#mean_points_team = gauss.trace["p_points_team"].mean()
#print(bets_of_team[49])
#bet = mean_wins_team_against * wins_of_team_against_enemy_in_last_five[49] +\
#      mean_points_team * points_of_team_in_last_five[49] +\
#      mean_wins_team * wins_of_team_in_last_five[49]
#print(bet)
#
#error = 0
#for i, value in enumerate(enemies_of_team):
#    bet = mean_wins_team_against * wins_of_team_against_enemy_in_last_five[i] +\
#      mean_wins_team * wins_of_team_in_last_five[i] +\
#      mean_points_team * points_of_team_in_last_five[i]
#    error += abs(bet - bets_of_team[i])
#avg_error = error / 304
#print(avg_error)